### Individual Assignment 4 - Pandas, Numpy, Sentiment Analysis 

### Background Info

WaterPro, an Australia-based company, sells a product called WaterCure. The company believes that the sales performance of WaterCure is affected by weather conditions. To remain profitable, WaterPro needs to achieve an average monthly sales revenue of at least $40,000. The company has provided a CSV file with daily sales data, including information on sales units, unit prices, and average daily temperatures.

In addition, WaterPro has been collecting customer reviews for WaterCure. With a $5 incentive per review, the company has gathered 500 reviews, and they want to understand customer sentiment. The company seeks to classify these reviews into positive, neutral, or negative categories to gain insight into customer satisfaction.

In [1]:
import pandas as pd
import spacy
import nltk

In [4]:
df= pd.read_csv ('daily_sales.csv',skiprows = 2)

**1. find out which months have the sales revenue below $ 40,000 and the average temperature of these months (please use Numpy package for this task)**

In [191]:
df.head()

,Date,Daily Units Sold,Daily Unit Price,Daily Temperature (C)
0,1/1/2024,91,24.0,25
1,1/2/2024,90,24.0,24
2,1/3/2024,70,24.0,19
3,1/4/2024,89,24.0,23
4,1/5/2024,100,24.0,36


In [193]:
# Convert the 'Date' column to datetime format for easy handling
df['Date'] = pd.to_datetime(df['Date'])

# Extract month and year from the Date column
df['YearMonth'] = df['Date'].dt.to_period('M')

df['Revenue'] = df['Daily Units Sold'] * df['Daily Unit Price']

In [195]:
# Group by YearMonth to calculate monthly sales revenue and average temperature
monthly_data = df.groupby('YearMonth').agg(
    Monthly_Sales=('Revenue', 'sum'),
    Avg_Temperature=('Daily Temperature (C)', 'mean')
).reset_index()

In [205]:
monthly_data

,YearMonth,Monthly_Sales,Avg_Temperature
0,2024-01,66168.0,25.387097
1,2024-02,56752.5,24.103448
2,2024-03,53372.0,22.709677
3,2024-04,45795.0,20.033333
4,2024-05,42693.0,18.548387
5,2024-06,35793.0,15.433333
6,2024-07,35137.0,13.838710
7,2024-08,22200.0,16.225806


In [201]:
# Filter the months where sales are below $40,000 directly using pandas
below_40k = monthly_data[monthly_data['Monthly_Sales'] < 40000]

In [203]:
below_40k

,YearMonth,Monthly_Sales,Avg_Temperature
5,2024-06,35793.0,15.433333
6,2024-07,35137.0,13.838710
7,2024-08,22200.0,16.225806


**2. print out this information for the company.**

In [207]:
for index, row in below_40k.iterrows():
    print(f"Month: {row['YearMonth']}, Sales Revenue: ${row['Monthly_Sales']}, Average Temperature: {row['Avg_Temperature']:.2f}°C")

Month: 2024-06, Sales Revenue: $35793.0, Average Temperature: 15.43°C
Month: 2024-07, Sales Revenue: $35137.0, Average Temperature: 13.84°C
Month: 2024-08, Sales Revenue: $22200.0, Average Temperature: 16.23°C


**3. label each review as positive, neutral, or negative based on its compound score with SentimentIntensityAnalyzer and add a new column to the data frame calling it "sentiment"**

In [223]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

In [7]:
df = pd.read_csv('water_product_reviews_500_actual_names (1).csv')

In [9]:
df.head()

,Reviewer Name,Review Date,Rating,Review Text
0,Owen White,2023-01-01,3,"The product meets basic needs, but that's abou..."
1,Bob Jenkins,2023-01-02,5,"Very satisfied with this product, excellent qu..."
2,Rachel Harris,2023-01-03,1,This product didn't meet my expectations; I wa...
3,Olivia Clark,2023-01-04,1,"I had issues with bottles leaking, very dissat..."
4,Violet Thompson,2023-01-05,1,This product didn't meet my expectations; I wa...


In [227]:
# Initialize SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [275]:
# Function to calculate sentiment based on the compound score
def get_sentiment(review):
    score = sia.polarity_scores(review)['compound']
    if score >= 0.05:
        return 'positive'
    elif score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

In [277]:
# Apply the sentiment analysis to each review and create a new column 'sentiment'
df['sentiment'] = df['Review Text'].apply(get_sentiment)

In [279]:
# Display the dataframe with the new sentiment column
print(df.head())

     Reviewer Name Review Date  Rating  \
0       Owen White  2023-01-01       3   
1      Bob Jenkins  2023-01-02       5   
2    Rachel Harris  2023-01-03       1   
3     Olivia Clark  2023-01-04       1   
4  Violet Thompson  2023-01-05       1   

                                         Review Text sentiment YearMonth  
0  The product meets basic needs, but that's abou...   neutral   2023-01  
1  Very satisfied with this product, excellent qu...  positive   2023-01  
2  This product didn't meet my expectations; I wa...  negative   2023-01  
3  I had issues with bottles leaking, very dissat...  negative   2023-01  
4  This product didn't meet my expectations; I wa...  negative   2023-01  


**4. calculate the average rating of the reviews for each review category (i.e., positive, neutral, and negative)**

In [243]:
# Group by the 'sentiment' column and calculate the average rating for each group
average_ratings = df.groupby('sentiment')['Rating'].mean().reset_index()

In [245]:
# Rename columns for clarity
average_ratings.columns = ['Sentiment', 'Average Rating']
# Display the results
print(average_ratings)

  Sentiment  Average Rating
0  negative        1.451923
1   neutral        3.000000
2  positive        4.187500


**5. summarize the total number of positive, neutral, and negative reviews for each month**

In [247]:
# Ensure the Date column is in datetime format
df['Review Date'] = pd.to_datetime(df['Review Date'])

In [249]:
# Extract YearMonth from the Review Date
df['YearMonth'] = df['Review Date'].dt.to_period('M')

In [251]:
# Group by YearMonth and sentiment, and count the number of reviews in each group
monthly_sentiment_counts = df.groupby(['YearMonth', 'sentiment']).size().unstack(fill_value=0).reset_index()

In [253]:
# Display the result
print(monthly_sentiment_counts)

sentiment YearMonth  negative  neutral  positive
0           2023-01        13        3        15
1           2023-02        14        4        10
2           2023-03        10        3        18
3           2023-04         6        2        22
4           2023-05        19        0        12
5           2023-06        11        1        18
6           2023-07        17        3        11
7           2023-08        13        2        16
8           2023-09        19        0        11
9           2023-10         9        3        19
10          2023-11        11        1        18
11          2023-12        16        2        13
12          2024-01        16        1        14
13          2024-02        11        2        16
14          2024-03        10        4        17
15          2024-04        10        0        20
16          2024-05         3        5         6


**6. calculate the average rating of the reviews for each month**

In [255]:
# Ensure the 'Review Date' is in datetime format
df['Review Date'] = pd.to_datetime(df['Review Date'])

# Extract YearMonth from the Review Date
df['YearMonth'] = df['Review Date'].dt.to_period('M')

In [267]:
# Group by YearMonth and calculate the average rating for each month
monthly_average_ratings = df.groupby('YearMonth')['Rating'].mean().reset_index()

# Rename columns for clarity
monthly_average_ratings.columns = ['Month', 'Average Rating']

In [269]:
# Display the result
print(monthly_average_ratings)

      Month  Average Rating
0   2023-01        2.870968
1   2023-02        2.785714
2   2023-03        3.290323
3   2023-04        3.600000
4   2023-05        2.387097
5   2023-06        3.133333
6   2023-07        2.741935
7   2023-08        3.096774
8   2023-09        2.400000
9   2023-10        3.258065
10  2023-11        3.233333
11  2023-12        2.419355
12  2024-01        2.741935
13  2024-02        3.034483
14  2024-03        3.129032
15  2024-04        3.233333
16  2024-05        3.142857


In [271]:
df.head()

,Reviewer Name,Review Date,Rating,Review Text,sentiment,YearMonth
0,Owen White,2023-01-01,3,"The product meets basic needs, but that's abou...",neutral,2023-01
1,Bob Jenkins,2023-01-02,5,"Very satisfied with this product, excellent qu...",positive,2023-01
2,Rachel Harris,2023-01-03,1,This product didn't meet my expectations; I wa...,negative,2023-01
3,Olivia Clark,2023-01-04,1,"I had issues with bottles leaking, very dissat...",negative,2023-01
4,Violet Thompson,2023-01-05,1,This product didn't meet my expectations; I wa...,negative,2023-01


**7. write your data frame that has the sentiment column to a csv file called "YourLastName_YourFirstName_sentiment.csv"**

In [273]:
df.to_csv('Hsieh_Luna_sentiment.csv', index=False)

### Summary

* Monthly Sales Revenue and Temperature: The monthly sales revenue below `$40,000` and corresponding average temperatures were identified, giving an understanding of how external conditions like temperature might influence customer purchases. Specifically, the sales revenue for months below `$40,000` ranged from `$22,200` to `$35,793` with average temperatures between 13.84°C and 16.23°C. This suggests that external factors, such as temperature, could play a role in influencing sales performance.

* Sentiment Analysis of Reviews: Reviews were categorized into positive, neutral, and negative based on the compound score from the Sentiment Intensity Analyzer, providing a breakdown of customer sentiment for WaterPro’s product. For example, the top 10 lowest-rated reviews often mention product leaks or unmet expectations, while the top 10 highest-rated reviews praise the product’s value and taste.

* Average Rating by Sentiment: For each sentiment category, the average rating was calculated, offering insight into how ratings align with customer sentiments. Positive reviews had an average rating of 4.19, while negative reviews averaged at 1.45, showing a clear correlation between sentiment and rating.

* Review Distribution by Month and Sentiment: The total number of positive, neutral, and negative reviews was summarized for each month, providing a temporal view of customer feedback trends. The number of reviews ranged from 0 to 19 for any given sentiment in a month.

* Monthly Average Ratings: The average rating for each month was calculated to assess overall customer satisfaction over time. Fluctuations in average ratings by month indicate that customer satisfaction varies, with notable dips in months where negative reviews spiked.

### Interpretation

* Sales and External Factors: The months with sales below $40,000 often coincided with either warmer or cooler temperatures. This suggests that customer behavior may be influenced by environmental factors, especially if the product is seasonal or climate-sensitive.

* Customer Sentiment Analysis: The sentiment analysis reveals that a significant proportion of reviews are either neutral or negative, despite some positive feedback. This suggests there may be aspects of the product or service that are not meeting customer expectations, leading to dissatisfaction or ambivalence.

* Average Rating by Sentiment: The ratings strongly correlate with the sentiment categories, as expected. Positive reviews tend to have high ratings, while negative reviews have lower ones. Neutral reviews are generally accompanied by mid-range ratings, indicating moderate satisfaction or mixed opinions.

* Trends in Customer Feedback: The distribution of sentiment across months may show seasonal patterns in customer satisfaction or reflect periods when product issues or improvements occurred. For example, a spike in negative reviews in May and September 2023 could signal product flaws or poor service during those periods, which may require targeted investigation.

* Monthly Ratings: The fluctuations in average ratings by month indicate that customer satisfaction varies over time. Identifying trends in these ratings can help WaterPro understand when and why customer satisfaction dips, which could inform product or service improvements.